# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --jsonArray establishments.json
`

In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created

databases = mongo.list_database_names()
print(databases)

['admin', 'config', 'local', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
db = mongo.uk_food

# List collections in the uk_food database
collections = db.list_collection_names()
print(collections)


['establishments']


In [7]:
# review the collections in our new database
db = mongo["uk_food"]

collections = db.list_collection_names()

# Print the list of collections
print(collections)

['establishments']


In [8]:
# review a document in the establishments collection

establishments_collection = db["establishments"]

# Find and display one document from the establishments collection
document = establishments_collection.find_one()

# Print the document
print(document)


{'_id': ObjectId('65f8dc50f66ab12e097a9210'), 'FHRSID': 289353, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002468', 'BusinessName': 'St Marys COE (aided) Primary School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'St Marys Church Of England Primary School', 'AddressLine2': 'Warren Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6QH', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2016-09-13T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'}, 'RightToReply': '', 'Distance': 4591.871474174837, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extrac

In [9]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "address": {
        "street": "123 Main Street",
        "city": "Greenwich",
        "postcode": "ABC123"
    },
    "cuisine": "Halal",
    "rating": "Not rated"
}


In [11]:
# Insert the new restaurant into the collection
result = establishments_collection.insert_one(new_restaurant)

# Print the inserted document's ID
print("Inserted document ID:", result.inserted_id)


Inserted document ID: 65fa1d42256bb1ee48d9ba3a


In [12]:
# Check that the new restaurant was inserted
newly_inserted_document = establishments_collection.find_one({"name": "Penang Flavours"})

# Print the document
print(newly_inserted_document)


{'_id': ObjectId('65f8e2b66b944f1922ff91d4'), 'name': 'Penang Flavours', 'address': {'street': '123 Main Street', 'city': 'Greenwich', 'postcode': 'ABC123'}, 'cuisine': 'Halal', 'rating': 'Not rated', 'BusinessTypeID': 1}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query_filter = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Project only the BusinessTypeID and BusinessType fields
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Find the document matching the query filter and project only the specified fields
result = establishments_collection.find_one(query_filter, projection)

# Print the result
print(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID

query_filter = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Project only the BusinessTypeID field
projection = {"_id": 0, "BusinessTypeID": 1}

# Find the document matching the query filter and project only the BusinessTypeID field
business_type_document = establishments_collection.find_one(query_filter, projection)

# Get the BusinessTypeID from the document
business_type_id = business_type_document["BusinessTypeID"]

# Update the new restaurant document with the correct BusinessTypeID
update_query = {"name": "Penang Flavours"}
update_values = {"$set": {"BusinessTypeID": business_type_id}}

# Perform the update operation
update_result = establishments_collection.update_one(update_query, update_values)

# Check if the update was successful
if update_result.modified_count > 0:
    print("New restaurant updated with the correct BusinessTypeID.")
else:
    print("Failed to update the new restaurant.")


Failed to update the new restaurant.


In [15]:
# Confirm that the new restaurant was updated

updated_restaurant_document = establishments_collection.find_one({"name": "Penang Flavours"})

# Print the updated document
print(updated_restaurant_document)


{'_id': ObjectId('65f8e2b66b944f1922ff91d4'), 'name': 'Penang Flavours', 'address': {'street': '123 Main Street', 'city': 'Greenwich', 'postcode': 'ABC123'}, 'cuisine': 'Halal', 'rating': 'Not rated', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Find how many documents have LocalAuthorityName as "Dover"

query_filter = {"LocalAuthorityName": "Dover"}

# Count the number of documents matching the query filter
documents_count = establishments_collection.count_documents(query_filter)

# Print the number of documents
print("Number of documents with LocalAuthorityName as 'Dover':", documents_count)


Number of documents with LocalAuthorityName as 'Dover': 0


In [17]:
# Delete all documents where LocalAuthorityName is "Dover"

query_filter = {"LocalAuthorityName": "Dover"}

# Delete all documents matching the query filter
delete_result = establishments_collection.delete_many(query_filter)

# Print the number of documents deleted
print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 0


In [18]:
# Check if any remaining documents include Dover
remaining_documents_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})

# Print the number of remaining documents with "Dover"
print("Number of remaining documents with LocalAuthorityName as 'Dover':", remaining_documents_count)


Number of remaining documents with LocalAuthorityName as 'Dover': 0


In [19]:
# Check that other documents remain with 'find_one'

document = establishments_collection.find_one()

# Print the document
print(document)


{'_id': ObjectId('65f8dc50f66ab12e097a9210'), 'FHRSID': 289353, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002468', 'BusinessName': 'St Marys COE (aided) Primary School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'St Marys Church Of England Primary School', 'AddressLine2': 'Warren Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6QH', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2016-09-13T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'}, 'RightToReply': '', 'Distance': 4591.871474174837, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extrac

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Change the data type from String to Decimal for longitude and latitude

update_result = establishments_collection.update_many(
    {},
    [
        {"$set": {"Latitude": {"$toDouble": "$Latitude"}}},
        {"$set": {"Longitude": {"$toDouble": "$Longitude"}}}
    ]
)

# Print the number of documents updated
print("Number of documents updated:", update_result.modified_count)


Number of documents updated: 38788


Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Change the data type from String to Integer for RatingValue

update_result = establishments_collection.update_many(
    {"RatingValue": {"$exists": True}},  # Filter for documents where RatingValue exists
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Convert RatingValue to integer
)

# Print the number of documents updated
print("Number of documents updated to change RatingValue data type to integer:", update_result.modified_count)


Number of documents updated to change RatingValue data type to integer: 34694


In [26]:
# Check that the coordinates and rating value are now numbers
for results in db.establishments.find({}, ['geocode.longitude', 'geocode.latitude', 'RatingValue']):
    pprint(results)

{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9210'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'}}
{'RatingValue': 4,
 '_id': ObjectId('65f8dc50f66ab12e097a9214'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9215'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9216'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'}}
{'RatingValue': 4,
 '_id': ObjectId('65f8dc50f66ab12e097a9217'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9218'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9219'),
 'geocode': {'latitude': '51.0783519967076', 'longitude': '1.18590330311705'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a921a'),
 'geocode': {'latitud

 'geocode': {'latitude': '51.379418', 'longitude': '1.38391'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9881'),
 'geocode': {'latitude': '51.3872146606445', 'longitude': '1.3899199962616'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9882'),
 'geocode': {'latitude': '51.389355', 'longitude': '1.391458'}}
{'RatingValue': 4,
 '_id': ObjectId('65f8dc50f66ab12e097a9883'),
 'geocode': {'latitude': '51.389355', 'longitude': '1.391458'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9884'),
 'geocode': {'latitude': '51.387487', 'longitude': '1.389825'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9885'),
 'geocode': {'latitude': '51.389355', 'longitude': '1.391458'}}
{'RatingValue': 4,
 '_id': ObjectId('65f8dc50f66ab12e097a9886'),
 'geocode': {'latitude': '51.389355', 'longitude': '1.391458'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc50f66ab12e097a9887'),
 'geocode': {'latitude': '51.387233', 'longitude': '1.389878'}}
{'RatingValue': 5,
 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



: 5,
 '_id': ObjectId('65f8dc52f66ab12e097b2574'),
 'geocode': {'latitude': '51.756135', 'longitude': '0.102122'}}
{'RatingValue': None,
 '_id': ObjectId('65f8dc52f66ab12e097b2575'),
 'geocode': {'latitude': '51.8354454040527', 'longitude': '0.181565999984741'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc52f66ab12e097b2576'),
 'geocode': {'latitude': '51.6885388', 'longitude': '0.0322583'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc52f66ab12e097b2577'),
 'geocode': {'latitude': '51.774094', 'longitude': '0.118524'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc52f66ab12e097b2578'),
 'geocode': {'latitude': '51.774095', 'longitude': '0.118525'}}
{'RatingValue': 5,
 '_id': ObjectId('65f8dc52f66ab12e097b2579'),
 'geocode': {'latitude': '51.768775', 'longitude': '0.112319'}}
{'RatingValue': None,
 '_id': ObjectId('65f8dc52f66ab12e097b257a'),
 'geocode': {'latitude': '51.768775', 'longitude': '0.112319'}}
{'RatingValue': None,
 '_id': ObjectId('65f8dc52f66ab12e097b257b'),
 'geocode': {'latit